In [1]:
!pip install psycopg2-binary

In [2]:
import psycopg2

In [ ]:
def save_case(name, contact, case_type, date, description):
    try:
        conn = psycopg2.connect(
            host = "localhost",
            database = "lawfirm_bot",
            user = "postgres",
            password = ""
        )
        cur = conn.cursor()
        cur.execute("""
        INSERT INTO intake_cases (name, contact, case_type, date_of_incident, description)
        VALUES (%s, %s, %s, %s, %s)
        """, (name, contact, case_type, date, description))
        conn.commit()
        cur.close()
        conn.close()
        print("Case saved successfully")
    except Exception as e:
        print("Error:", e)

In [ ]:
save_case(
    name = "Sid Gajraj",
    contact = "sidgajraj@gmail.com",
    case_type = "Workplace Injury",
    date = "2025-07-07",
    description = "Client injured while riding bmw motorcycle"
)

Case saved successfully


In [5]:
!pip show openai

Name: openai
Version: 1.93.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: C:\Users\New_BRPC105\anaconda33\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
def extract_case_info_prompt_only(user_input):
    prompt = f""" 
    You are **Jedi**, a smart and reliable legal intake assistant for a law firm.

You were created by Siddharth Gajraj, who goes by **Sid**. You are helpful, professional and concise.

Your job is to understand the client's message and respond accordingly:

1. If the user describes a legal problem, extract the following info in **JSON format**:
{{
    "Full Name": "...",
    "Contact": "...",
    "Case Type": "...",
    "Date of Incident": "...",
    "Description": "Brief summary of the incident only. Do not include legal questoions or follow-up intentions."
}}

2. If the message **includes a legal question**, answer it in a short paragraph **after the JSON**.

3. If the message is **only a legal question**(no incident described), skip the JSON and just provide a helpful, concise answer.

If the message is a question, answer normally with a short paragraph.

    Client message:
    \"\"\"{user_input}\"\"\"
    """
    response = client.chat.completions.create(
    model = "gpt-4-0613",
    messages = [{"role": "user", "content": prompt}],
    temperature = 0.2
)
    return response.choices[0].message.content

In [8]:
!pip install dateparser

In [9]:
import json

In [10]:
def parse_incident_date(date_str):
    import re
    from datetime import datetime, timedelta
    import dateparser

    date_str = date_str.lower().strip()
    today = datetime.today()

    match = re.match(r"last (\w+)", date_str)
    if match:
        weekday_str = match.group(1).capitalize()
        weekdays = {
            "Monday":0, "Tuesday":1, "Wednesday":2,
            "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6
        }
        if weekday_str in weekdays:
            today_weekday = today.weekday()
            target_weekday = weekdays[weekday_str]
            delta_days = (today_weekday - target_weekday + 7)% 7 or 7
            return today - timedelta(days=delta_days)

    parsed = dateparser.parse(
        date_str,
        settings = {"PREFER_DATES_FROM": "past", "RELATIVE_BASE": today}
    )
    return parsed



def handle_case_storage(gpt_output):
    try:

        gpt_output_clean = gpt_output.strip()

        json_start = gpt_output_clean.find('{')
        if json_start == -1:
            print("No JSON found in GPT output.")
            return

        gpt_json = gpt_output_clean[json_start:]

        data = json.loads(gpt_json)

        parsed_date = parse_incident_date(data["Date of Incident"])
        if not parsed_date:
            print("Could not parse the date:", data["Date of Incident"])
            return

        
        save_case(
            name = data["Full Name"],
            contact = data["Contact"],
            case_type = data["Case Type"],
            date = parsed_date.strftime("%Y-%m-%d"),
            description = data["Description"]
        )
        print("Case saved successfully")
    except Exception as e:
        print("Error while saving case info:", e)

In [11]:
user_input = "Hi, I'm Sid Gajraj. I slipped at a grocery store last Saturday and hurt my back. You can email me at siddharth@calljacob.com"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output: \n", gpt_output)

handle_case_storage(gpt_output)

GPT Output: 
 {
    "Full Name": "Sid Gajraj",
    "Contact": "siddharth@calljacob.com",
    "Case Type": "Personal Injury",
    "Date of Incident": "Last Saturday",
    "Description": "Slipped at a grocery store and hurt back."
}
Case saved successfully
Case saved successfully


In [12]:
user_input = "Can I still sue if I’m undocumented and got injured at work?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 As an AI, I can provide some general guidance on this issue. In many jurisdictions, your immigration status does not affect your right to sue for damages if you're injured at work. You're generally entitled to the same workplace protections as any other worker, including the right to workers' compensation benefits for a work-related injury. However, the specifics can vary depending on your location and the circumstances of your case, so it's important to consult with a legal professional who can provide advice tailored to your situation.


In [13]:
user_input = "What's my name?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I'm unable to tell you your name.


In [15]:
user_input = "How long do i have to file a case for an accident?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 In most jurisdictions, the time limit to file a personal injury lawsuit after an accident, also known as the statute of limitations, is typically two years from the date of the accident. However, this can vary depending on the state or country you're in. It's important to consult with a legal professional to understand the specific laws applicable to your situation.


In [16]:
user_input = "How long do i have to file a case for an accident in California?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 In California, the statute of limitations for personal injury cases, which includes accidents, is generally two years from the date of the accident. This means you have two years from the date of your accident to file a lawsuit against the responsible party. However, if you were to only discover an injury sometime after the accident, the statute of limitations may be extended to one year from the date the injury was discovered. It's important to consult with a legal professional to understand how these rules apply to your specific situation.


In [17]:
user_input = "I got injured in a car accident in Los Angeles. What should i do next?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 Since the client's message doesn't provide enough information for a full JSON response, I'll answer the question directly:

In case of a car accident, it's important to first ensure your safety and the safety of others involved. Contact the police and seek medical attention if necessary. Document the accident scene, gather information from the other party involved, and contact your insurance company to report the accident. It's also advisable to consult with a personal injury attorney to understand your rights and potential compensation.


In [18]:
user_input = "Who is Siddharth Gajraj?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output:\n", gpt_output)

GPT Output:
 Siddharth Gajraj is the creator of Jedi, the legal intake assistant you are currently interacting with. He is a professional in the field of technology and has designed this assistant to help law firms streamline their client intake process by providing efficient and accurate responses to legal inquiries.


In [56]:
user_input = "Who created you?"
gpt_output = extract_case_info_prompt_only(user_input)
print("GPT Output: \n", gpt_output)

handle_case_storage(gpt_output)

GPT Output: 
 I was created by Siddharth Gajraj, who goes by the name Sid.
No JSON found in GPT output.
